In [1]:
import spacy

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash
python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 79.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
2022-07-21 02:23:58.073892: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-21 02:23:58.073929: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [3]:
nlp = spacy.load("en_core_web_sm")

In [7]:
doc = nlp("Time flies like an arrow")
tags = []
words = []
for token in doc:
    tags.append(token.pos_)
for token in doc:
    words.append(token.text)
tags

['NOUN', 'VERB', 'ADP', 'DET', 'NOUN']

G= (T, N, R,S)

Conjunto de terminales: T= {time, flies, arrow, an, like}

Conjunto de no terminales: N= {S, NP, VP, PP, Det, Nominal, Verb, Preposition, Noun}

Conjunto de reglas:

 R={

 S → NP VP				0.800

 NP → time | flies | arrow		0.002

 NP → Det Nominal			0.300

 NP → Nominal Nominal		0.200

 Nominal → time | flies | arrow	0.002

 Nominal → Nominal Noun		0.100

 Nominal → Nominal PP		0.200

 VP → time				0.004

 VP → flies| like			0.008

 VP → Verb NP				0.300

 VP → Verb PP				0.200

 PP → Preposition NP			0.100

 Verb → time				0.010

 Verb → flies| like			0.020

 Noun → time | flies | arrow		0.010

 Det → an				0.050

 Preposition → like			0.050

 }
 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=16bbac0e-c0cf-475a-9977-7067c93e2eaf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>